In [6]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import time

In [7]:
df=pd.read_csv('twitter_data_with_dow_for_analysis.csv',parse_dates=['created_at'],index_col=0)

In [8]:
for x in df.columns:
    print(x)

text
created_at
retweet_count
favorite_count
trade_topic
china_topic
financial_topic
labor_topic
industry_topic
currency_rates_topic
general_related_topic
budget_topic
foreign_policy_topic
covid_topic
sensitive_topic
environment_topic
sentiment
month
1_minutes_forward_pct_change
5_minutes_forward_pct_change
10_minutes_forward_pct_change
15_minutes_forward_pct_change
30_minutes_forward_pct_change
1_hours_forward_pct_change
3_hours_forward_pct_change
6_hours_forward_pct_change
1_minutes_forward_vol_sum
5_minutes_forward_vol_sum
10_minutes_forward_vol_sum
15_minutes_forward_vol_sum
30_minutes_forward_vol_sum
1_hours_forward_vol_sum
3_hours_forward_vol_sum
0
Facebook
Instagram
Media Studio
Mobile Web (M5)
Neatly For BlackBerry 10
Periscope
TweetDeck
TwitLonger Beta
Twitlonger
Twitter Ads
Twitter Media Studio
Twitter Mirror for iPad
Twitter QandA
Twitter Web App
Twitter Web Client
Twitter for Android
Twitter for BlackBerry
Twitter for Websites
Twitter for iPad
Twitter for iPhone
Vine - Make

In [9]:
X=df.loc[:,['created_at',
'Twitter for iPhone',
'Twitter for Android',
'favorite_count',
'retweet_count',
'trade_topic',
'china_topic',
'financial_topic',
'labor_topic',
'industry_topic',
'currency_rates_topic',
'general_related_topic',
'budget_topic',
'foreign_policy_topic',
'covid_topic',
'sensitive_topic',
'environment_topic',
'sentiment',
'text'
]].copy()

In [10]:
y=df[['1_minutes_forward_pct_change',
'5_minutes_forward_pct_change',
'10_minutes_forward_pct_change',
'15_minutes_forward_pct_change',
'30_minutes_forward_pct_change',
'1_hours_forward_pct_change',
'3_hours_forward_pct_change',
'6_hours_forward_pct_change',
'1_minutes_forward_vol_sum',
'5_minutes_forward_vol_sum',
'10_minutes_forward_vol_sum',
'15_minutes_forward_vol_sum',
'30_minutes_forward_vol_sum',
'1_hours_forward_vol_sum',
'3_hours_forward_vol_sum']].copy()

In [11]:
inspect_df=pd.concat([X.loc[~y['15_minutes_forward_vol_sum'].isna()],y['15_minutes_forward_vol_sum'].dropna()],axis=1)

In [87]:
inspect_df=inspect_df.loc[inspect_df['created_at']>='2016-05-03 22:00:00'].copy()

In [88]:
for column in inspect_df.iloc[:,:-1].columns:
    inspect_df[column] = inspect_df[[column, 'sentiment']].product(axis=1)
# inspect_df.drop('sentiment',1,inplace=True)

In [89]:
inspect_df.drop(['created_at','text'],1,inplace=True)

In [90]:
# inspect_df.insert(0, 'financial_topic_sentiment_interaction',
#                              inspect_df[["sentiment", 'financial_topic']].product(axis=1))
# inspect_df.insert(0, 'sensitive_topic_sentiment_interaction',
#                              inspect_df[["sentiment", 'sensitive_topic']].product(axis=1))

In [91]:
# inspect_df=inspect_df.replace([np.inf, -np.inf], float('nan'))
inspect_df=inspect_df.dropna()

In [92]:
mod = sm.OLS(inspect_df.iloc[:,-1],inspect_df.iloc[:,:-1])
res = mod.fit()

In [93]:
print(res.summary())

                               OLS Regression Results                              
Dep. Variable:     1_hours_forward_vol_sum   R-squared:                       0.000
Model:                                 OLS   Adj. R-squared:                 -0.002
Method:                      Least Squares   F-statistic:                  0.009865
Date:                     Wed, 24 Jun 2020   Prob (F-statistic):               1.00
Time:                             07:45:48   Log-Likelihood:                -48997.
No. Observations:                     8246   AIC:                         9.803e+04
Df Residuals:                         8229   BIC:                         9.815e+04
Df Model:                               17                                         
Covariance Type:                 nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [ ]:
from sklearn.linear_model import Ridge
rr = Ridge(alpha=0.00000000001,normalize=False)
rr.fit(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1])
w = rr.coef_

In [ ]:
print(rr.score(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1]))

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.000000001,normalize=False)
clf.fit(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1])
print(clf.score(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1]))

In [ ]:
coefs=pd.DataFrame(columns=['variable','coefficients'])

In [ ]:
coefs['variable']=inspect_df.iloc[:,:-1].columns

In [ ]:
coefs['coefficients'] = clf.coef_

In [ ]:
len(coefs[coefs['coefficients']!=0])

In [ ]:
for i,x in coefs[coefs['coefficients']!=0].sort_values('coefficients').iterrows():
    print(x['variable'])